# Start Guide for the devicely package

# Install devicely

Installing devicely is as easy as executing `pip install devicely`.

To run this notebook, get the data by cloning [this repository](https://github.com/jostmorgenstern/devicely-documentation-sample-data) in the same directory as this notebook.

In [1]:
import os

import devicely
import pandas as pd
pd.options.mode.chained_assignment = None

base_path = 'devicely-documentation-sample-data'

# Empatica E4

The Empatice E4 wristband can be used to obtain data from inter-beat intervals, electrodermal activity, heart rate, temperature and blood volume pulse. The wristband uses [this directory structure](https://github.com/jostmorgenstern/devicely-documentation-sample-data/tree/main/Empatica) for its measurement data. The `tags.csv` file contains the timestamps of important events and is optional. Only if the remaining csv files are present the empatica reader can be created.

### Read the data

Create an EmpaticaReader object:

In [2]:
empatica_reader = devicely.EmpaticaReader(os.path.join(base_path, 'Empatica'))

Access the sampling frequencies and starting times for all signals:

In [3]:
empatica_reader.start_times

{'ACC': Timestamp('2020-10-16 17:04:29'),
 'BVP': Timestamp('2020-10-16 17:04:29'),
 'EDA': Timestamp('2020-10-16 17:04:29'),
 'HR': Timestamp('2020-10-16 17:04:39'),
 'TEMP': Timestamp('2020-10-16 17:04:29'),
 'IBI': Timestamp('2020-10-16 17:04:29')}

In [4]:
empatica_reader.sample_freqs

{'ACC': 32.0, 'BVP': 64.0, 'EDA': 4.0, 'HR': 1.0, 'TEMP': 4.0}

Access the individual dataframes via the attributes ACC, BVP, EDA, HR, TEMP, IBI and tags:

In [5]:
empatica_reader.HR.head()

2020-10-16 17:04:39    51.00
2020-10-16 17:04:40    51.50
2020-10-16 17:04:41    51.00
2020-10-16 17:04:42    52.75
2020-10-16 17:04:43    64.00
Name: HR, dtype: float64

Access a joined dataframe of all signals:

In [6]:
empatica_reader.data.head()

,ACC_X,ACC_Y,ACC_Z,BVP,EDA,HR,TEMP
2020-10-16 17:04:29.000000,-3,11,60,-0.0,0.0,NaN,31.13
2020-10-16 17:04:29.015625,NaN,NaN,NaN,-0.0,NaN,NaN,NaN
2020-10-16 17:04:29.031250,-3,11,60,-0.0,NaN,NaN,NaN
2020-10-16 17:04:29.046875,NaN,NaN,NaN,-0.0,NaN,NaN,NaN
2020-10-16 17:04:29.062500,-2,11,60,-0.0,NaN,NaN,NaN


The dataframe contains nan values because the individual signals have different sampling frequencies.


### Anonymize the data:

Apply a timeshift:

In [7]:
empatica_reader.timeshift()
empatica_reader.start_times

{'ACC': Timestamp('2019-12-30 22:46:33.745538704'),
 'BVP': Timestamp('2019-12-30 22:46:33.745538704'),
 'EDA': Timestamp('2019-12-30 22:46:33.745538704'),
 'HR': Timestamp('2019-12-30 22:46:43.745538704'),
 'TEMP': Timestamp('2019-12-30 22:46:33.745538704'),
 'IBI': Timestamp('2019-12-30 22:46:33.745538704')}

By providing no parameter to `timeshift` the data is shifted by a random time interval between one month and two years to the past. You can also provide a `pandas.Timedelta` object to shift the data by that timedelta or a `pandas.Timestamp` object to shift your data such that this timestamp is the earliest entry.

### Write the data:

In [8]:
empatica_write_path = os.path.join(base_path, 'Empatica_write_dir')
empatica_reader.write(empatica_write_path)
os.listdir(empatica_write_path)

['tags.csv', 'IBI.csv', 'EDA.csv', 'TEMP.csv', 'ACC.csv', 'BVP.csv', 'HR.csv']

## SpaceLabs Monitoring System

SpaceLabs uses [a single file](https://github.com/jostmorgenstern/devicely-documentation-sample-data/blob/main/Spacelabs/spacelabs.abp) to output metadata as well as the actual signals.

### Read the data

Create a `SpacelabsReader` object:

In [9]:
spacelabs_reader = devicely.SpacelabsReader(os.path.join(base_path, 'Spacelabs', 'spacelabs.abp'))

Acess the metadata:

In [10]:
spacelabs_reader.subject

'001V0'

In [11]:
spacelabs_reader.metadata

{'PATIENTINFO': {'DOB': None, 'RACE': None},
 'REPORTINFO': {'PHYSICIAN': None,
  'NURSETECH': 'admin',
  'STATUS': 'NOTCONFIRMED',
  'CALIPERSUMMARY': {'COUNT': '0'}}}

Access the signal dataframe:

In [12]:
spacelabs_reader.data.head()

,timestamp,date,time,SYS(mmHg),DIA(mmHg),ACC_x,ACC_y,ACC_z,error
0,2019-03-01 16:18:00,2019-03-01,16:18:00,107,76,78.0,78.0,NaN,NaN
1,2019-03-01 16:19:00,2019-03-01,16:19:00,96,62,63.0,63.0,NaN,NaN
2,2019-03-01 16:22:00,2019-03-01,16:22:00,100,68,64.0,64.0,NaN,NaN
3,2019-03-01 16:23:00,2019-03-01,16:23:00,103,68,68.0,68.0,NaN,NaN
4,2019-03-01 16:25:00,2019-03-01,16:25:00,101,67,65.0,65.0,NaN,NaN


### Drop erroneous data entries

The data contains an `error` column which identifies erroneous entries. You can eliminate these entries with the `SpacelabsReader`:

In [13]:
spacelabs_reader.drop_EB()
spacelabs_reader.data.head()

,date,time,SYS(mmHg),DIA(mmHg),ACC_x,ACC_y,ACC_z,error
timestamp,,,,,,,,
2019-03-01 16:18:00,2019-03-01,16:18:00,107,76,78.0,78.0,NaN,NaN
2019-03-01 16:19:00,2019-03-01,16:19:00,96,62,63.0,63.0,NaN,NaN
2019-03-01 16:22:00,2019-03-01,16:22:00,100,68,64.0,64.0,NaN,NaN
2019-03-01 16:23:00,2019-03-01,16:23:00,103,68,68.0,68.0,NaN,NaN
2019-03-01 16:25:00,2019-03-01,16:25:00,101,67,65.0,65.0,NaN,NaN


Before dropping the errors, the data may contain duplicate timestamps. After dropping, the timestamps uniquely identify the data entries. For this reason the timestamp column is used as the index after the `drop_EB` call.

### Anonymize the data:

Apply a timeshift:

In [14]:
spacelabs_reader.timeshift()
spacelabs_reader.data.head()

,date,time,SYS(mmHg),DIA(mmHg),ACC_x,ACC_y,ACC_z,error
timestamp,,,,,,,,
2018-05-12 18:28:00,2018-05-12,18:28:00,107,76,78.0,78.0,NaN,NaN
2018-05-12 18:29:00,2018-05-12,18:29:00,96,62,63.0,63.0,NaN,NaN
2018-05-12 18:32:00,2018-05-12,18:32:00,100,68,64.0,64.0,NaN,NaN
2018-05-12 18:33:00,2018-05-12,18:33:00,103,68,68.0,68.0,NaN,NaN
2018-05-12 18:35:00,2018-05-12,18:35:00,101,67,65.0,65.0,NaN,NaN


By providing no parameter to `timeshift` the data is shifted by a random time interval between one month and two years to the past. You can also provide a `pandas.Timedelta` object to shift the data by that timedelta or a `pandas.Timestamp` object to shift your data such that this timestamp is the earliest entry.

### Set a time window:

Setting a time window means adding two columns, `window_start` and `window_end` to the data. Depending on the `window_type` parameter, the window will either be before, after or around the timestamp column. Look at the module reference for more information about this method. 

In [15]:
spacelabs_reader.set_window(pd.Timedelta(seconds=30), "bfill")
spacelabs_reader.data.head()

,date,time,SYS(mmHg),DIA(mmHg),ACC_x,ACC_y,ACC_z,error,window_start,window_end
timestamp,,,,,,,,,,
2018-05-12 18:28:00,2018-05-12,18:28:00,107,76,78.0,78.0,NaN,NaN,2018-05-12 18:27:30,2018-05-12 18:28:00
2018-05-12 18:29:00,2018-05-12,18:29:00,96,62,63.0,63.0,NaN,NaN,2018-05-12 18:28:30,2018-05-12 18:29:00
2018-05-12 18:32:00,2018-05-12,18:32:00,100,68,64.0,64.0,NaN,NaN,2018-05-12 18:31:30,2018-05-12 18:32:00
2018-05-12 18:33:00,2018-05-12,18:33:00,103,68,68.0,68.0,NaN,NaN,2018-05-12 18:32:30,2018-05-12 18:33:00
2018-05-12 18:35:00,2018-05-12,18:35:00,101,67,65.0,65.0,NaN,NaN,2018-05-12 18:34:30,2018-05-12 18:35:00


## Bittium Faros

The Faros device outpus data in [EDF files](https://www.edfplus.info/specs/edf.html). These are specifically made for health sensor data and not human-readable.

### Read the data:

In [16]:
faros_reader = devicely.FarosReader(os.path.join(base_path, 'Faros', 'faros.EDF'))

Access metadata:

In [17]:
faros_reader.start_time

Timestamp('2019-03-01 16:12:43')

In [18]:
faros_reader.sample_freqs

{'ECG': 1000.0, 'ACC': 100.0, 'Marker': 1.0, 'HRV': 5.0}

In [19]:
faros_reader.units

{'ECG': 'uV', 'ACC': 'mg', 'HRV': 'ms'}

You can access the individual signals via the `ECG`, `ACC`, `HRV` and `Marker` attributes:

In [20]:
faros_reader.ACC.head()

,X,Y,Z,mag
2019-03-01 16:12:43.000,164.0,23.0,-1172.0,1183.642260
2019-03-01 16:12:43.010,152.0,23.0,-1172.0,1182.039339
2019-03-01 16:12:43.020,152.0,-24.0,-1079.0,1089.917887
2019-03-01 16:12:43.030,117.0,11.0,-985.0,991.985383
2019-03-01 16:12:43.040,-47.0,246.0,-1125.0,1152.540672


Access a joined dataframe of all signals:

In [21]:
faros_reader.data.head()

,ECG,ACC_X,ACC_Y,ACC_Z,ACC_mag,Marker,HRV
2019-03-01 16:12:43.000,26.0,164.0,23.0,-1172.0,1183.64226,0.0,0.0
2019-03-01 16:12:43.001,-6.0,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-01 16:12:43.002,-31.0,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-01 16:12:43.003,-39.0,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-01 16:12:43.004,-17.0,NaN,NaN,NaN,NaN,NaN,NaN


### Anonymize the data

Apply a timeshift:

In [22]:
faros_reader.timeshift()
faros_reader.data.head()

,ECG,ACC_X,ACC_Y,ACC_Z,ACC_mag,Marker,HRV
2018-07-31 03:19:41.000,26.0,164.0,23.0,-1172.0,1183.64226,0.0,0.0
2018-07-31 03:19:41.001,-6.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-07-31 03:19:41.002,-31.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-07-31 03:19:41.003,-39.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-07-31 03:19:41.004,-17.0,NaN,NaN,NaN,NaN,NaN,NaN


By providing no parameter to `timeshift` the data is shifted by a random time interval between one month and two years to the past. You can also provide a `pandas.Timedelta` object to shift the data by that timedelta or a `pandas.Timestamp` object to shift your data such that this timestamp is the earliest entry.

### Write the data

You can write back the data in the original EDF format or to a directory of individual signal files. Writing to a directory is the preferred method. You can find out why this is the case in our module reference.

In [23]:
faros_write_path = os.path.join(base_path, 'Faros_write')
faros_reader.write(faros_write_path)
os.listdir(faros_write_path)

['ACC.csv', 'ECG.csv', 'HRV.csv', 'Marker.csv', 'meta.json']

You can also create a FarosReader from a written directory:

In [24]:
new_faros_reader = devicely.FarosReader(faros_write_path)
new_faros_reader.data.head()

,ECG,ACC_X,ACC_Y,ACC_Z,ACC_mag,Marker,HRV
2019-03-01 16:12:43.000,26.0,164.0,23.0,-1172.0,1183.64226,0.0,0.0
2019-03-01 16:12:43.001,-6.0,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-01 16:12:43.002,-31.0,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-01 16:12:43.003,-39.0,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-01 16:12:43.004,-17.0,NaN,NaN,NaN,NaN,NaN,NaN


## Biovotion Everion

The Everion device outputs data in [multiple csv files](https://github.com/jostmorgenstern/devicely-documentation-sample-data/tree/main/Everion). Each csv file has a `tag` column which specifies the type of measurement. You can see the different tags and what they mean by looking at `EverionReader.SIGNAL_TAGS`, `EverionReader.SENSOR_TAGS` and `EverionReader.FEATURE_TAGS`.

In [25]:
devicely.EverionReader.FEATURE_TAGS

{14: 'inter_pulse_interval',
 17: 'pis',
 18: 'pid',
 77: 'inter_pulse_deviation',
 78: 'pis_quality',
 79: 'pid_quality'}

### Read the data

In [26]:
everion_reader = devicely.EverionReader(os.path.join(base_path, 'Everion'))

If you would like to specify which tags to keep, you can specify this when initializing the reader.

Access the individual dataframes via aggregates, analytics_events, attributes_dailys, everion_events, features, sensors, signals attributes:

In [27]:
everion_reader.signals.head()

,count,streamType,tag,time,values,quality
0,806132,2,71,2019-03-01 15:39:58,0.000000,NaN
1,806132,2,13,2019-03-01 15:39:58,21.864220,100.0
2,806132,2,6,2019-03-01 15:39:58,65.000000,85.0
3,806132,2,66,2019-03-01 15:39:58,1.568628,NaN
4,806132,2,12,2019-03-01 15:39:58,18.000000,93.0


Access a joined dataframe of all signals:

In [28]:
everion_reader.data.head()

,heart_rate,heart_rate_quality,oxygen_saturation,oxygen_saturation_quality,heart_rate_variability,heart_rate_variability_quality,respiration_rate,respiration_rate_quality,ctemp,ctemp_quality,...,inter_pulse_interval,inter_pulse_interval_deviation,led1_data,led2_data,led3_data,led4_data,accx_data,accy_data,accz_data,acc_mag
time,,,,,,,,,,,,,,,,,,,,,
2019-03-01 13:23:05.000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3028.0,2989.0,2924.0,3340.0,368.0,2096.0,-3536.0,4126.976617
2019-03-01 13:23:05.019607808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3309.0,3336.0,3231.0,3417.0,256.0,2016.0,-3808.0,4316.324362
2019-03-01 13:23:05.039215872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3454.0,3443.0,3390.0,3492.0,144.0,2288.0,-3728.0,4376.489918
2019-03-01 13:23:05.058823680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3207.0,3217.0,3126.0,3543.0,96.0,2352.0,-3600.0,4301.292829
2019-03-01 13:23:05.078431488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3079.0,3092.0,2994.0,3538.0,0.0,2384.0,-3904.0,4574.349353


### Anonymize the data

Apply a timeshift:

In [29]:
everion_reader.timeshift()
everion_reader.data.head()

,heart_rate,heart_rate_quality,oxygen_saturation,oxygen_saturation_quality,heart_rate_variability,heart_rate_variability_quality,respiration_rate,respiration_rate_quality,ctemp,ctemp_quality,...,inter_pulse_interval,inter_pulse_interval_deviation,led1_data,led2_data,led3_data,led4_data,accx_data,accy_data,accz_data,acc_mag
time,,,,,,,,,,,,,,,,,,,,,
2018-05-17 03:02:41.000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3028.0,2989.0,2924.0,3340.0,368.0,2096.0,-3536.0,4126.976617
2018-05-17 03:02:41.019607808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3309.0,3336.0,3231.0,3417.0,256.0,2016.0,-3808.0,4316.324362
2018-05-17 03:02:41.039215872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3454.0,3443.0,3390.0,3492.0,144.0,2288.0,-3728.0,4376.489918
2018-05-17 03:02:41.058823680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3207.0,3217.0,3126.0,3543.0,96.0,2352.0,-3600.0,4301.292829
2018-05-17 03:02:41.078431488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3079.0,3092.0,2994.0,3538.0,0.0,2384.0,-3904.0,4574.349353


By providing no parameter to `timeshift` the data is shifted by a random time interval between one month and two years to the past. You can also provide a `pandas.Timedelta` object to shift the data by that timedelta or a `pandas.Timestamp` object to shift your data such that this timestamp is the earliest entry.

### Write the data

Write the data to a directory while keeping the same format as the original. If you used only a subset of tags when initializing the reader, only these tags will be written.

In [30]:
everion_write_path = os.path.join(base_path, 'Everion_write')
everion_reader.write(everion_write_path)
os.listdir(everion_write_path)

['everion_events.csv',
 'signals.csv',
 'attributes_dailys.csv',
 'features.csv',
 'aggregates.csv',
 'sensor_data.csv',
 'analytics_events.csv']

## Shimmer

Shimmer uses a [single CSV file](https://github.com/jostmorgenstern/devicely-documentation-sample-data/blob/main/Shimmer/shimmer.csv), indexed by time of measurement.

### Read the data

In [31]:
shimmer_reader = devicely.ShimmerPlusReader(os.path.join(base_path, 'Shimmer', 'shimmer.csv'))
shimmer_reader.data.head()

,Shimmer_40AC_Timestamp_Unix_CAL,Shimmer_40AC_Accel_LN_X_CAL,Shimmer_40AC_Accel_LN_Y_CAL,Shimmer_40AC_Accel_LN_Z_CAL,Shimmer_40AC_Accel_WR_X_CAL,Shimmer_40AC_Accel_WR_Y_CAL,Shimmer_40AC_Accel_WR_Z_CAL,Shimmer_40AC_Battery_CAL,Shimmer_40AC_Ext_Exp_A15_CAL,Shimmer_40AC_GSR_Range_CAL,...,Shimmer_40AC_Gyro_X_CAL,Shimmer_40AC_Gyro_Y_CAL,Shimmer_40AC_Gyro_Z_CAL,Shimmer_40AC_Int_Exp_A12_CAL,Shimmer_40AC_Mag_X_CAL,Shimmer_40AC_Mag_Y_CAL,Shimmer_40AC_Mag_Z_CAL,Shimmer_40AC_Pressure_BMP280_CAL,Shimmer_40AC_Temperature_BMP280_CAL,Shimmer_40AC_Accel_LN_mag
0,2020-07-28 10:56:50.034,-1.434783,10.0,0.554348,-3.930580,8.421305,-1.620586,4139.194139,1684.981685,2.0,...,0.137405,1.877863,-0.183206,1680.586081,-0.112444,-0.916042,-0.047976,100.435379,33.365878,10.117604
1,2020-07-28 10:56:50.057,-1.402174,10.0,0.554348,-3.923399,8.442849,-1.599042,4137.728938,1673.260073,2.0,...,0.183206,1.328244,-0.412214,1703.296703,-0.109445,-0.913043,-0.047976,100.429731,33.365878,10.113031
2,2020-07-28 10:56:50.074,-1.434783,10.0,0.554348,-3.897068,8.428486,-1.603830,4111.355311,1901.098901,2.0,...,0.274809,1.282443,-0.198473,1687.179487,-0.107946,-0.910045,-0.049475,100.441027,33.365878,10.117604
3,2020-07-28 10:56:50.099,-1.413043,10.0,0.521739,-3.932974,8.421305,-1.589467,4140.659341,1722.344322,2.0,...,0.229008,1.450382,-0.122137,1650.549451,-0.106447,-0.901049,-0.050975,100.441027,33.365878,10.112809
4,2020-07-28 10:56:50.111,-1.445652,10.0,0.510870,-3.944943,8.428486,-1.661281,4134.798535,1678.388278,2.0,...,0.137405,1.511450,-0.473282,1701.831502,-0.113943,-0.904048,-0.037481,100.438203,33.365878,10.116862


### Anonymize the data

Apply a timeshift:

In [32]:
shimmer_reader.timeshift()
shimmer_reader.data.head()

,Shimmer_40AC_Timestamp_Unix_CAL,Shimmer_40AC_Accel_LN_X_CAL,Shimmer_40AC_Accel_LN_Y_CAL,Shimmer_40AC_Accel_LN_Z_CAL,Shimmer_40AC_Accel_WR_X_CAL,Shimmer_40AC_Accel_WR_Y_CAL,Shimmer_40AC_Accel_WR_Z_CAL,Shimmer_40AC_Battery_CAL,Shimmer_40AC_Ext_Exp_A15_CAL,Shimmer_40AC_GSR_Range_CAL,...,Shimmer_40AC_Gyro_X_CAL,Shimmer_40AC_Gyro_Y_CAL,Shimmer_40AC_Gyro_Z_CAL,Shimmer_40AC_Int_Exp_A12_CAL,Shimmer_40AC_Mag_X_CAL,Shimmer_40AC_Mag_Y_CAL,Shimmer_40AC_Mag_Z_CAL,Shimmer_40AC_Pressure_BMP280_CAL,Shimmer_40AC_Temperature_BMP280_CAL,Shimmer_40AC_Accel_LN_mag
0,2020-03-10 03:00:42.974,-1.434783,10.0,0.554348,-3.930580,8.421305,-1.620586,4139.194139,1684.981685,2.0,...,0.137405,1.877863,-0.183206,1680.586081,-0.112444,-0.916042,-0.047976,100.435379,33.365878,10.117604
1,2020-03-10 03:00:42.997,-1.402174,10.0,0.554348,-3.923399,8.442849,-1.599042,4137.728938,1673.260073,2.0,...,0.183206,1.328244,-0.412214,1703.296703,-0.109445,-0.913043,-0.047976,100.429731,33.365878,10.113031
2,2020-03-10 03:00:43.014,-1.434783,10.0,0.554348,-3.897068,8.428486,-1.603830,4111.355311,1901.098901,2.0,...,0.274809,1.282443,-0.198473,1687.179487,-0.107946,-0.910045,-0.049475,100.441027,33.365878,10.117604
3,2020-03-10 03:00:43.039,-1.413043,10.0,0.521739,-3.932974,8.421305,-1.589467,4140.659341,1722.344322,2.0,...,0.229008,1.450382,-0.122137,1650.549451,-0.106447,-0.901049,-0.050975,100.441027,33.365878,10.112809
4,2020-03-10 03:00:43.051,-1.445652,10.0,0.510870,-3.944943,8.428486,-1.661281,4134.798535,1678.388278,2.0,...,0.137405,1.511450,-0.473282,1701.831502,-0.113943,-0.904048,-0.037481,100.438203,33.365878,10.116862


By providing no parameter to `timeshift` the data is shifted by a random time interval between one month and two years to the past. You can also provide a `pandas.Timedelta` object to shift the data by that timedelta or a `pandas.Timestamp` object to shift your data such that this timestamp is the earliest entry.

### Write the data

In [33]:
shimmer_reader.write(os.path.join(base_path, 'Shimmer', 'shimmer_write.csv'))

## Tags

You can use the TagReader to read data created by the Android app TimeStamp. Researches use this app to mark important times during experiments. The format simple, as can be seen in this [example file](https://github.com/jostmorgenstern/devicely-documentation-sample-data/blob/main/Tags/tags.csv).

### Read the data

In [34]:
tag_reader = devicely.TagReader(os.path.join(base_path, 'Tags', 'tags.csv'))
tag_reader.data.head()

,tag_number,tag
time,,
2019-03-01 16:16:37,1,Shake
2019-03-01 16:17:43,2,Start
2019-03-01 16:18:20,3,BP Measurement
2019-03-01 16:19:51,4,BP Measurement
2019-03-01 16:22:00,5,BP Measurement


### Anonymize the data

Apply a timeshift:

In [35]:
tag_reader.timeshift()
tag_reader.data.head()

,tag_number,tag
time,,
2018-02-26 15:05:54,1,Shake
2018-02-26 15:07:00,2,Start
2018-02-26 15:07:37,3,BP Measurement
2018-02-26 15:09:08,4,BP Measurement
2018-02-26 15:11:17,5,BP Measurement


By providing no parameter to `timeshift` the data is shifted by a random time interval between one month and two years to the past. You can also provide a `pandas.Timedelta` object to shift the data by that timedelta or a `pandas.Timestamp` object to shift your data such that this timestamp is the earliest entry.

### Write the data

In [36]:
tag_write_path = os.path.join(base_path, 'Tags', 'tags_write.csv')
tag_reader.write(tag_write_path)